In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import random

%config InlineBackend.figure_format = 'retina'

nlp = spacy.load('el_core_news_md')
df = pd.read_csv('../data/greek_fake_news.csv')
df.replace(to_replace='[\n\r\t]', value=' ', regex=True, inplace=True)

In [2]:
def load_data(train_data, limit=0, split=0.8):
    random.shuffle(train_data)
    train_data = train_data[-limit:]
    texts, labels = zip(*train_data)
    cats = [{"REAL": not bool(y), "FAKE": bool(y)} for y in labels]
    split = int(len(train_data) * split)
    
    return (texts[:split], cats[:split]), (texts[split:], cats[split:])

def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 0.0  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 0.0  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if label == "FAKE":
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.0
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.0
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if (precision + recall) == 0:
        f_score = 0.0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)
    return {"textcat_p": precision, "textcat_r": recall, "textcat_f": f_score}


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    100 non-null    object
 1   text     100 non-null    object
 2   source   100 non-null    object
 3   url      100 non-null    object
 4   is_fake  100 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 4.0+ KB


In [4]:
textcat=nlp.create_pipe( "textcat", config={"exclusive_classes": True, "architecture": "simple_cnn"})
nlp.add_pipe(textcat, last=True)
nlp.pipe_names



['tagger', 'parser', 'ner', 'textcat']

In [5]:
textcat.add_label("REAL")
textcat.add_label("FAKE")

1

In [6]:
df['tuples'] = df.apply(lambda row: (row['text'], row['is_fake']), axis=1)
train = df['tuples'].tolist()

In [7]:
(train_texts, train_cats), (dev_texts, dev_cats) = load_data(train, split=0.8)

train_data = list(zip(train_texts,[{'cats': cats} for cats in train_cats]))


In [8]:
from spacy.util import minibatch, compounding

n_iter = 20
# Disabling other components
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
with nlp.disable_pipes(*other_pipes):  # only train textcat
    optimizer = nlp.begin_training()

    print("Training the model...")
    print('{:^5}\t{:^5}\t{:^5}\t{:^5}'.format('LOSS', 'P', 'R', 'F'))

    # Performing training
    for i in range(n_iter):
        losses = {}
        batches = minibatch(train_data, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                       losses=losses)

      # Calling the evaluate() function and printing the scores
        with textcat.model.use_params(optimizer.averages):
            scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
        print('{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}'  
              .format(losses['textcat'], scores['textcat_p'],
                      scores['textcat_r'], scores['textcat_f']))

Training the model...
LOSS 	  P  	  R  	  F  
0.629	0.667	1.000	0.800
0.427	0.800	1.000	0.889
0.176	0.800	1.000	0.889
0.155	0.800	1.000	0.889
0.116	0.800	1.000	0.889
0.092	0.786	0.917	0.846
0.072	0.800	1.000	0.889
0.032	0.800	1.000	0.889
0.040	0.857	1.000	0.923
0.024	0.857	1.000	0.923
0.131	0.923	1.000	0.960
0.119	0.857	1.000	0.923
0.057	0.857	1.000	0.923
0.067	0.800	1.000	0.889
0.000	0.923	1.000	0.960
0.000	0.857	1.000	0.923
0.000	0.857	1.000	0.923
0.000	0.857	1.000	0.923
0.000	0.857	1.000	0.923
0.000	0.857	1.000	0.923


In [23]:
test = '''
30 Αυγούστου 2019. Ένα μυστηριώδες διαστημικό σκάφος περιστρέφεται γύρω από τη Γη τις τελευταίες 720 ημέρες. Τι ακριβώς είναι και βρίσκεται σε τροχιά για περίπου δύο χρόνια;

Αυτή είναι η πέμπτη αποστολή του διαστημικού αεροσκάφους X-37B, ενός στρατιωτικού σκάφους χωρίς αστροναύτες που κινείται με ηλιακή ενέργεια. 

Η αποστολή του Τροχιακού Δοκιμαστικού Οχήματος (Orbital Test Vehicle 5 – OTV-5) ξεκίνησε στις 7 Σεπτεμβρίου 2017 με τη βοήθεια ενός πυραύλου SpaceX Falcon 9.
Στην τέταρτη αποστολή το OTV-4, παρέμεινε σε τροχιά για 717 ημέρες, 20 ώρες και 42 λεπτά, τα οποία πλέον έχει ξεπεράσει το OTV-5, χωρίς λεπτομέρειες σχετικά με το πότε η αποστολή του μπορεί να λήξει.

Οι στόχοι της αποστολής του διαστημικού αεροσκάφους – παρόμοιο στο σχεδιασμό με τα διαστημικά λεωφορεία της NASA – παρέμειναν ένα μυστικό που φυλάσσεται πολύ καλά. Η Πολεμική Αεροπορία είναι πολύ φειδωλή στις δηλώσεις της, αφήνοντας τους ανθρώπους να υποθέσουν ότι είναι κάποιο είδος κατασκοπευτικού δορυφόρου, ενώ πολλές άλλες θεωρίες εμφανίζονται

Όταν το σκάφος ξεκίνησε, διάφοροι συνωμοσιογράφοι πρότειναν ότι θα μπορούσε να είναι ένα είδος “διαστημικού βομβιστή“, πολύ πριν ο Trump δημιουργήσει την λεγόμενη Space Force. Η θεωρία αυτή απορρίφθηκε πολύ γρήγορα από ειδικούς, οι οποίοι επεσήμαναν πως η αλλαγή του τροχιακού σκάφους ενός διαστημικού οχήματος απαιτεί πολύ ώθηση που θα τελείωνε γρήγορα την περιορισμένη παροχή καυσίμων του

Μια άλλη θεωρία από το περιοδικό Spaceflight είναι ότι πρόκειται για ένα κατασκοπευτικό σκάφος:

“Η επιτήρηση του διαστήματος από το διάστημα, είναι ένα ολοκαίνουργιο παιχνίδι οπότε το X-37B μπορεί να χρησιμοποιείται για να παρακολουθεί στενά τον εκκολαπτόμενο διαστημικό σταθμό της Κίνας”, δήλωσε ο διευθυντής του Spaceflight, Δρ Ο David Baker στο BBC News το 2012, αν και αυτό θεωρήθηκε απίθανο, βάσει της τροχιάς του.

Αλλά η επίσημη εξήγηση της Πολεμικής Αεροπορίας δεν δίνει πάρα πολλά:

“Το τροχιακό όχημα X-37B, ή το OTV, είναι ένα πειραματικό πρόγραμμα δοκιμών για την επίδειξη τεχνολογιών για μια αξιόπιστη, επαναχρησιμοποιούμενη, μη επανδρωμένη πλατφόρμα δοκιμής διαστήματος, για την Πολεμική Αεροπορία των ΗΠΑ”, δήλωσε η Πολεμική Αεροπορία.

«Οι πρωταρχικοί στόχοι του X-37B είναι διπλοί: επαναχρησιμοποιήσιμες τεχνολογίες διαστημικών οχημάτων για το μέλλον της Αμερικής στο διάστημα και πειράματα λειτουργίας που μπορούν να επιστραφούν και να εξερευνηθούν στη Γη.

The #X37B #OTV4 is the newest and most advanced re-entry spacecraft. Find out more about today's landing here: https://t.co/GUGgOMQiYg pic.twitter.com/HfHHVnWhYc

— U.S. Air Force (@usairforce) May 7, 2017
“Οι τεχνολογίες που δοκιμάζονται στο πρόγραμμα περιλαμβάνουν προηγμένες οδηγίες καθοδήγησης, πλοήγησης και ελέγχου, συστήματα θερμικής προστασίας, αεροηλεκτρονικής, δομές και ασπίδες υψηλής θερμοκρασίας, συμβατική επαναχρησιμοποιήσιμη μόνωση, ελαφριά ηλεκτρομηχανολογικά συστήματα πτήσης, προηγμένα συστήματα πρόωσης, προηγμένα υλικά και αυτόνομη τροχιακή πτήση, προσγείωση.”

Προς το παρόν, θα πρέπει είτε να αποδεχθείτε την επίσημη εξήγηση είτε απλώς να είστε άνετοι με το γεγονός ότι υπάρχει ένα μυστηριώδες διαστημικό σκάφος που πετάει ψηλά πάνω από το κεφάλι μας και μόνο ο στρατός των ΗΠΑ ξέρει γιατί.
'''

doc = nlp(test)
doc.cats

{'REAL': 0.0027723785024136305, 'FAKE': 0.9972276091575623}